In [1]:
pip install sqlite-vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 4.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sqlite3
import sqlite_vec

db = sqlite3.connect(":memory:")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

vec_version, = db.execute("select vec_version()").fetchone()
print(f"vec_version={vec_version}")

vec_version=v0.1.1


In [3]:
import sqlite3
import json
import sqlite_vec

# Connect to SQLite database
conn = sqlite3.connect('arxiv_papers.db')

conn.enable_load_extension(True)

# Register the extension with SQLite
sqlite_vec.load(conn)

cursor = conn.cursor()

# Create a table with columns to match the JSON structure
cursor.execute('''
CREATE TABLE IF NOT EXISTS papers (
    id TEXT PRIMARY KEY,
    authors TEXT,
    title TEXT,
    abstract TEXT,
    update_date TEXT
)
''')

# Create virtual tables for embeddings using vec0
cursor.execute('''
CREATE VIRTUAL TABLE IF NOT EXISTS title_embeddings USING vec0(title_embedding FLOAT[384])
''')

cursor.execute('''
CREATE VIRTUAL TABLE IF NOT EXISTS abstract_embeddings USING vec0(abstract_embedding FLOAT[384])
''')

conn.commit()

In [4]:
import sqlite3
import json

# Connect to SQLite database
conn = sqlite3.connect('arxiv_papers.db')
cursor = conn.cursor()

# Load JSON data
with open('/kaggle/input/blog-number1/papers_dataset.json', 'r') as file:
    data = json.load(file)

# Prepare the insert statements
papers_insert = '''
INSERT OR REPLACE INTO papers (id, authors, title, abstract, update_date)
VALUES (?, ?, ?, ?, ?)
'''



# Insert data
for paper in data:
    # Insert into papers table
    cursor.execute(papers_insert, (
        paper['id'],
        paper['authors'],
        paper['title'],
        paper['abstract'],
        paper['update_date']
    ))
    
    

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Data insertion complete.")


Data insertion complete.


In [5]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect('/kaggle/working/arxiv_papers.db')

# Query to select data from the papers table
query = "SELECT * FROM papers LIMIT 10;"

# Execute the query and load the result into a DataFrame
df = pd.read_sql_query(query, conn)

# Display the DataFrame
df


,id,authors,title,abstract,update_date
0,0707.3559,Wilson Wong,Practical Approach to Knowledge-based Question...,This research hypothesized that a practical ...,2007-07-25
1,1602.04983,"Sreyasi Nag Chowdhury, Mateusz Malinowski, And...",Contextual Media Retrieval Using Natural Langu...,The widespread integration of cameras in han...,2016-02-17
2,1709.03968,"Nabiha Asghar, Pascal Poupart, Jesse Hoey, Xin...",Affective Neural Response Generation,Existing neural conversational models proces...,2017-09-14
3,2001.00471,"Kennedy Ralston, Yuhao Chen, Haruna Isah, Farh...",A Voice Interactive Multilingual Student Suppo...,Systems powered by artificial intelligence a...,2020-01-03
4,2002.00747,"Maartje ter Hoeve, Robert Sim, Elnaz Nouri, Ad...",Conversations with Documents. An Exploration o...,The role of conversational assistants has be...,2020-02-04
5,2004.12316,"Peixiang Zhong, Chen Zhang, Hao Wang, Yong Liu...",Towards Persona-Based Empathetic Conversationa...,Empathetic conversational models have been s...,2020-11-20
6,2007.15211,Victor Dibia,NeuralQA: A Usable Library for Question Answer...,Existing tools for Question Answering (QA) h...,2020-12-01
7,2008.03717,"Antonios Minas Krasakis, Mohammad Aliannejadi,...",Analysing the Effect of Clarifying Questions o...,Recent research on conversational search hig...,2020-08-12
8,2008.08247,"Kun Zhou, Wayne Xin Zhao, Hui Wang, Sirui Wang...",Leveraging Historical Interaction Data for Imp...,"Recently, conversational recommender system ...",2020-08-20
9,2012.07619,"Maartje ter Hoeve, Julia Kiseleva, Maarten de ...",What Makes a Good and Useful Summary? Incorpor...,Automatic text summarization has enjoyed gre...,2022-05-02


In [6]:
!pip install sentence_transformers datasets beir faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 52.4 MB/s eta 0:00:00:00:0100:01
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63550 sha256=ca4dc575161552430617d45bea31b84c03e25a727113bc648e8724f59b5d0069
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=293148 sha256=994ab4e646a1b80f5ed53409434f8c2060ba72302ec89cc1696fc4d49cfa705c
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built beir pytrec_eval


In [7]:
import sqlite3
import json
from sentence_transformers import SentenceTransformer
import numpy as np
import sqlite_vec

# Load the model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
model = model.half()  # Use fp16 for faster computation

# Connect to SQLite database
conn = sqlite3.connect('arxiv_papers.db')
conn.enable_load_extension(True)
sqlite_vec.load(conn)
cursor = conn.cursor()

# Recreate the virtual tables with the correct dimensions
cursor.execute('DROP TABLE IF EXISTS title_embeddings')
cursor.execute('DROP TABLE IF EXISTS abstract_embeddings')
cursor.execute('''
CREATE VIRTUAL TABLE title_embeddings USING vec0(title_embedding FLOAT[1024])
''')
cursor.execute('''
CREATE VIRTUAL TABLE abstract_embeddings USING vec0(abstract_embedding FLOAT[1024])
''')

# Function to create embeddings
def create_embedding(text):
    embedding = model.encode(text, convert_to_numpy=True, normalize_embeddings=True)
    return embedding

# Function to insert embeddings
def insert_embedding(table_name, rowid, embedding):
    embedding_json = json.dumps(embedding.tolist())
    cursor.execute(f"INSERT OR REPLACE INTO {table_name} (rowid, {table_name[:-10]}embedding) VALUES (?, ?)", 
                   (rowid, embedding_json))

# Fetch all papers
cursor.execute("SELECT rowid, title, abstract FROM papers")
papers = cursor.fetchall()

# Process each paper
for rowid, title, abstract in papers:
    try:
        # Create and insert title embedding
        title_embedding = create_embedding(title)
        insert_embedding('title_embeddings', rowid, title_embedding)

        # Create and insert abstract embedding
        abstract_embedding = create_embedding(abstract)
        insert_embedding('abstract_embeddings', rowid, abstract_embedding)

        # Commit after each paper to save progress
        conn.commit()

        print(f"Processed paper {rowid}")
    except Exception as e:
        print(f"Error processing paper {rowid}: {e}")
        # Rollback the transaction in case of an error
        conn.rollback()

# Close the connection
conn.close()

print("Embedding creation and insertion complete.")


/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/171 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/114k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed paper 44


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Processed paper 45
Embedding creation and insertion complete.


In [9]:
import sqlite3
import sqlite_vec

# Connect to SQLite database
conn = sqlite3.connect('arxiv_papers.db')
conn.enable_load_extension(True)
sqlite_vec.load(conn)
cursor = conn.cursor()

# Check title embeddings
cursor.execute("SELECT rowid, title_embedding FROM title_embeddings LIMIT 5")
title_results = cursor.fetchall()
print("Title Embeddings:")
for row in title_results:
    print(f"Row ID: {row[0]}, Embedding: {row[1][:50]}...")  # Print a part of the embedding

# Check abstract embeddings
cursor.execute("SELECT rowid, abstract_embedding FROM abstract_embeddings LIMIT 5")
abstract_results = cursor.fetchall()
print("\nAbstract Embeddings:")
for row in abstract_results:
    print(f"Row ID: {row[0]}, Embedding: {row[1][:50]}...")  # Print a part of the embedding

# Close the connection
conn.close()


Title Embeddings:
Row ID: 1, Embedding: b'\x00\x80Q=\x00\x80\x15<\x00@\x81\xbd\x00\xa0\x0c\xbd\x00\x80\x04\xbd\x00@s=\x00\xa0I\xbc\x00@\xad\xbb\x00\x00j\xbc\x00\xc0\xd7<\x00@\xc7<\x00\xa0\xa0<\x00@'...
Row ID: 2, Embedding: b'\x00\xa0\x05=\x00\xa0\x8c<\x00\xc0_\xbc\x00\xa0i\xbc\x00\x80\x8e;\x00\x80\xad<\x00 *\xbd\x00\xe0\x07;\x00\xa0R\xbb\x00\x00\xac<\x00\xa0 \xbc\x00\x80\x1a\xbb\x00\xa0'...
Row ID: 3, Embedding: b'\x00 \x96<\x00\x80g=\x00\xc0,\xbd\x00\xa0\x9e\xbc\x00 \xe2\xbb\x00\xa0\xaa\xbd\x00\x80d\xbb\x00\xc0\xd8<\x00\xc0\x13\xbd\x00\x00\xff\xbb\x00`\xd9\xbc\x00\xc00\xb9\x00`'...
Row ID: 4, Embedding: b'\x00\xc04<\x00\x00\xe6\xbc\x00 \xcd\xbb\x00\x00\x1a\xbd\x00\xe0\xe8<\x00\xa0\x0e=\x00\x80R<\x00\xc0F\xbc\x00\xa0[;\x00`3=\x00`\x83\xbc\x00\x00\xac<\x00@'...
Row ID: 5, Embedding: b'\x00`(=\x00\xe0)\xbd\x00\x00\xbe\xb9\x00\xe0\xeb\xbb\x00\x80\x97;\x00\xc0\xcb<\x00\x00\xcd\xbc\x00\x80\x9f\xbb\x00\x00\x8e\xbb\x00\x80\x16=\x00\xc0\x08\xbc\x00\xe0g=\x00\x80'...

Abstract Embeddings:
Row 

In [ ]:
import sqlite3
import numpy as np
from sentence_transformers import SentenceTransformer
import sqlite_vec

# Load the model
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")
model = model.half()  # Use fp16 for faster computation

# Connect to SQLite database
conn = sqlite3.connect('arxiv_papers.db')
conn.enable_load_extension(True)
sqlite_vec.load(conn)
cursor = conn.cursor()

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def bytes_to_array(byte_data):
    return np.frombuffer(byte_data, dtype=np.float32)

def search_papers(query, top_k=5):
    # Create embedding for the query
    query_embedding = model.encode(query, convert_to_numpy=True, normalize_embeddings=True)

    # Fetch all papers with their embeddings
    cursor.execute('''
    SELECT p.id, p.title, p.abstract, p.authors, p.update_date,
           te.title_embedding, ae.abstract_embedding
    FROM papers p
    JOIN title_embeddings te ON p.rowid = te.rowid
    JOIN abstract_embeddings ae ON p.rowid = ae.rowid
    ''')
    papers = cursor.fetchall()

    # Calculate similarities
    results = []
    for paper in papers:
        title_embedding = bytes_to_array(paper[5])
        abstract_embedding = bytes_to_array(paper[6])
        title_similarity = cosine_similarity(query_embedding, title_embedding)
        abstract_similarity = cosine_similarity(query_embedding, abstract_embedding)
        max_similarity = max(title_similarity, abstract_similarity)
        results.append(paper[:5] + (max_similarity,))

    # Sort results by similarity score
    sorted_results = sorted(results, key=lambda x: x[5], reverse=True)[:top_k]

    return sorted_results

def print_results(results):
    for i, result in enumerate(results, 1):
        print(f"\nResult {i}:")
        print(f"ID: {result[0]}")
        print(f"Title: {result[1]}")
        print(f"Authors: {result[3]}")
        print(f"Update Date: {result[4]}")
        print(f"Abstract: {result[2][:200]}...")  # Print first 200 characters of abstract
        print(f"Similarity Score: {result[5]:.4f}")

def main():
    while True:
        query = input("\nEnter your search query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        results = search_papers(query)
        print_results(results)

    conn.close()

if __name__ == "__main__":
    main()


Enter your search query (or 'quit' to exit):  provide AI driven text summarization technique papers


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1:
ID: 2012.07619
Title: What Makes a Good and Useful Summary? Incorporating Users in Automatic
  Summarization Research
Authors: Maartje ter Hoeve, Julia Kiseleva, Maarten de Rijke
Update Date: 2022-05-02
Abstract:   Automatic text summarization has enjoyed great progress over the years and is
used in numerous applications, impacting the lives of many. Despite this
development, there is little research that mean...
Similarity Score: 0.7589

Result 2:
ID: cs/0005020
Title: Centroid-based summarization of multiple documents: sentence extraction,
  utility-based evaluation, and user studies
Authors: Dragomir R. Radev (University of Michigan), Hongyan Jing (Columbia
  University), Malgorzata Budzikowska (IBM TJ Watson Research Center)
Update Date: 2007-05-23
Abstract:   We present a multi-document summarizer, called MEAD, which generates
summaries using cluster centroids produced by a topic detection and tracking
system. We also describe two new techniques, based o...
Similarity S


Enter your search query (or 'quit' to exit):  Human in the loop related papers


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1:
ID: 2305.13068
Title: Making Language Models Better Tool Learners with Execution Feedback
Authors: Shuofei Qiao, Honghao Gui, Chengfei Lv, Qianghuai Jia, Huajun Chen,
  Ningyu Zhang
Update Date: 2024-03-15
Abstract:   Tools serve as pivotal interfaces that enable humans to understand and
reshape the environment. With the advent of foundation models, AI systems can
utilize tools to expand their capabilities and in...
Similarity Score: 0.6603

Result 2:
ID: 2403.13344
Title: USE: Dynamic User Modeling with Stateful Sequence Models
Authors: Zhihan Zhou, Qixiang Fang, Leonardo Neves, Francesco Barbieri, Yozen
  Liu, Han Liu, Maarten W. Bos, Ron Dotsch
Update Date: 2024-03-21
Abstract:   User embeddings play a crucial role in user engagement forecasting and
personalized services. Recent advances in sequence modeling have sparked
interest in learning user embeddings from behavioral d...
Similarity Score: 0.6446

Result 3:
ID: 2311.11701
Title: Control in Hybrid Chatbots
Authors: T


Enter your search query (or 'quit' to exit):  Optimizing search relevance with machine learning models.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Result 1:
ID: 2008.03717
Title: Analysing the Effect of Clarifying Questions on Document Ranking in
  Conversational Search
Authors: Antonios Minas Krasakis, Mohammad Aliannejadi, Nikos Voskarides,
  Evangelos Kanoulas
Update Date: 2020-08-12
Abstract:   Recent research on conversational search highlights the importance of
mixed-initiative in conversations. To enable mixed-initiative, the system
should be able to ask clarifying questions to the user...
Similarity Score: 0.7133

Result 2:
ID: 2211.00201
Title: CCS Explorer: Relevance Prediction, Extractive Summarization, and Named
  Entity Recognition from Clinical Cohort Studies
Authors: Irfan Al-Hussaini, Davi Nakajima An, Albert J. Lee, Sarah Bi, Cassie
  S. Mitchell
Update Date: 2023-02-02
Abstract:   Clinical Cohort Studies (CCS), such as randomized clinical trials, are a
great source of documented clinical research. Ideally, a clinical expert
inspects these articles for exploratory analysis ran...
Similarity Score: 0.6994

Result